In [1]:
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from models import create_fixed_input_shape_model, create_variable_input_shape_model

In [ ]:
IMAGE_SIZE = (160, 160)
NUM_CLASSES = 10
train_dir = "datasets/imagenette2/train"
test_dir = "datasets/imagenette2/val"

# __Training a Fixed Input Shape Model__

In [ ]:
data_augment_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_data_generator = data_augment_generator.flow_from_directory(
    train_dir, batch_size=32, class_mode="categorical", target_size=IMAGE_SIZE
)

data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

test_data_generator = data_generator.flow_from_directory(
    test_dir, batch_size=64, class_mode="categorical", target_size=IMAGE_SIZE
)

In [ ]:
model = create_fixed_input_shape_model(IMAGE_SIZE, NUM_CLASSES)
model.compile(
    optimizer="RMSProp",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [ ]:
history = model.fit(
    train_data_generator,
    validation_data=test_data_generator,
    epochs=20,
    verbose=1,
)

In [ ]:
model.save_weights("fixed-imagenette2.h5")

# __Training a Variable Input Shape Model__

In [ ]:
# import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img, array_to_img
from PIL import Image

In [ ]:
class VariableImageGenerator():
    def __init__(self, directory, preprocessing_fn, batch_size=16, shuffle=False, max_dimension=None):        
        self.directories = directory
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.max_dimension = max_dimension
        self.img_paths, self.class_labels = self._traverse(directory)
        self.preprocessing_fn = preprocessing_fn

        #index array for shuffling data
        self.idx = np.arange(len(self.img_paths))
    
    def __len__(self):
        #number of batches in an epoch
        return int(np.ceil(len(self.img_paths) / float(self.batch_size)))

    def _traverse(self, directory):
        img_paths = []
        class_labels = []

        #create list of image file paths and class target labels
        for class_label, class_dir in enumerate(sorted(os.listdir(directory))):
            img_paths += [os.path.join(directory, class_dir, fn) for fn in os.listdir(os.path.join(directory, class_dir))]
            class_labels += [class_label for _ in os.listdir(os.path.join(directory, class_dir))]

        class_labels = np.array(class_labels)
        class_one_hot_labels = np.zeros((class_labels.size, class_labels.max() + 1))
        class_one_hot_labels[np.arange(class_labels.size), class_labels] = 1

        return np.array(img_paths), class_one_hot_labels
    
    
    def _load_img(self, img_path):
        #load image from path and convert to array
        img = load_img(img_path, color_mode='rgb', interpolation='nearest', target_size=None)
        
        #downsample image if above allowed size if specified
        img_max_dimension = max(img.size) 
        if self.max_dimension:
            if img_max_dimension > self.max_dimension:
                new_dimension = tuple(dim * self.max_dimension // img_max_dimension for dim in img.size)
                img = img.resize(size=new_dimension, resample=Image.BILINEAR)
            
        img = img_to_array(img) # convert PIL image to numpy array

        return self.preprocessing_fn(img) # scale image values
    
    
    def _pad_img(self, img, shape):
        #pad images to match largest image in batch
        width, height = shape
        img_width, img_height, _ = img.shape

        diff_w = ((width - img_width) // 2, ((width - img_width) // 2) + ((width - img_width) % 2))
        diff_h = ((height - img_height) // 2, ((height - img_height) // 2) + ((height - img_height) % 2))

        return np.pad(img, (diff_w, diff_h, (0, 0)), mode='constant', constant_values=0.)

    def __call__(self):
        #shuffle index
        if self.shuffle:
            np.random.shuffle(self.idx)
        
        #generate batches
        for batch_no in range(len(self)):
            _img_paths = self.img_paths[self.idx[batch_no * self.batch_size:(batch_no + 1) * self.batch_size]]
            img_classes = self.class_labels[self.idx[batch_no * self.batch_size:(batch_no + 1) * self.batch_size]]

            imgs = [self._load_img(img_path) for img_path in _img_paths]
            batch_max_dim = tuple(max([img.shape[i] for img in imgs]) for i in range(2))
            imgs = np.array([self._pad_img(image, batch_max_dim) for image in imgs])

            yield imgs, img_classes

In [ ]:
train_generator = VariableImageGenerator(train_dir, preprocessing_fn=preprocess_input, batch_size=32, shuffle=True, max_dimension=160)
test_generator = VariableImageGenerator(test_dir, preprocessing_fn=preprocess_input, batch_size=32, max_dimension=160)

#convert generators into tf.data.Dataset objects for optimization with keras model fit method
train_dataset = Dataset.from_generator(train_generator, (tf.float32, tf.int32), (tf.TensorShape([None, None, None, 3]), tf.TensorShape([None, NUM_CLASSES])))
test_dataset = Dataset.from_generator(test_generator, (tf.float32, tf.int32), (tf.TensorShape([None, None, None, 3]), tf.TensorShape([None, NUM_CLASSES])))

In [ ]:
model = create_variable_input_shape_model(NUM_CLASSES)
model.compile(
    optimizer="RMSProp",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=test_data_generator,
    epochs=20,
    verbose=1,
)

In [ ]:
model.save_weights("variable-imagenette2.h5")